In [ ]:
# %run /home/ptenkaate/scratch/Master-Thesis/convert_ipynb_to_py_files.ipynb

In [ ]:
import torch
import torch.nn.functional as F

from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import Resize, Compose, ToTensor, Normalize

import argparse
import os
import math 
import skimage
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import PyQt5


import time
import pickle

from datetime import datetime
from pathlib import Path

from py_files.new_dataset import *

from py_files.cnn_model import *
from py_files.pigan_model import *

from py_files.seq_pi_gan_functions import *

# %matplotlib qt

#### Import classes

In [ ]:
DEVICE = set_device()

print('----------------------------------')
print('Using device for training:', DEVICE)
print('----------------------------------')

# Set the run you want to evaluate 

In [ ]:
sorted(os.listdir(path='saved_runs'))

In [ ]:
run = sorted(os.listdir(path='saved_runs'))[-1]
# run = "pi-gan 2021-06-05 00:58:36 "

ARGS = load_args(run, print_changed=False)

print(run)

for var, val in vars(ARGS).items():
    print(f"{var.ljust(15)} \t {val}")

# Evaluation

In [ ]:
ARGS.batch_size = 3

train_dl, val_dl, test_dl = initialize_dataloaders(ARGS)

#### Load models

In [ ]:
ARGS.print_models=False
models, optims, schedulers = load_models_and_optims(ARGS)

In [ ]:
# models["pcmra_siren"]

In [ ]:
ARGS.pretrained_best_dataset = "train"
ARGS.pretrained_best_loss = "mask"

load_pretrained_models(run, ARGS.pretrained_best_dataset, ARGS.pretrained_best_loss,
                       models, optims, pretrained_models=None)

In [ ]:
##### loss function #####
mask_criterion = nn.BCELoss()    
pcmra_criterion = nn.MSELoss()  
dice_criterion = calc_dice_loss

In [ ]:
def scroll_through_output(dataloader, shape=[24, 64, 64], transform=False):
    with torch.no_grad():
        pcmras = masks = loss_covers = mask_outs = pcmra_outs = torch.Tensor([])

        titles = []

        mask_losses, pcmra_losses, dice_losses = [], [], []

        for batch in dataloader: 

            if transform:
                batch = transform_batch(batch, ARGS)


            _, _, _, pcmra, coords_array, pcmra_array, mask_array, surface_array, norm_array = get_siren_batch(batch, ARGS)


            mask_out = get_complete_image(models, pcmra, coords_array, ARGS)
            pcmra_out = get_complete_image(models, pcmra, coords_array, ARGS, output="pcmra")

            mask_loss = mask_criterion(mask_out, mask_array) 
            dice_loss = dice_criterion(mask_out, mask_array) 
            pcmra_loss = pcmra_criterion(pcmra_out, pcmra_array) 

            mask_losses.append(mask_loss.item())
            dice_losses.append(dice_loss.item())
            pcmra_losses.append(pcmra_loss.item())

            for p, m, po, mo in zip(pcmra_array, mask_array, pcmra_out, mask_out):
                pcmras = torch.cat((pcmras, p.cpu().view(shape).detach().permute(1, 2, 0)), 2)
                masks = torch.cat((masks, m.cpu().view(shape).detach().permute(1, 2, 0)), 2)
                mask_outs = torch.cat((mask_outs, mo.cpu().view(shape).detach().permute(1, 2, 0)), 2)
                pcmra_outs = torch.cat((pcmra_outs, po.cpu().view(shape).detach().permute(1, 2, 0)), 2)

        
        print("Mask loss: ", np.array(mask_losses).mean())
        print("Dice loss: ", np.array(dice_losses).mean())
        
        print("PCMRA loss:", np.array(pcmra_losses).mean())

        titles = "Results"

        window = Show_images(titles, (pcmras.numpy(), "pcmras"), 
                                     (pcmra_outs.numpy(), "pcmra output"),
                                     (masks.numpy(), "masks"),
                                     (mask_outs.numpy(), "mask output"), 
#                                      (loss_covers.numpy(), "loss_covers"),
                            )

        return window

In [ ]:
ARGS.rotate = True
ARGS.translate = True
ARGS.flip = True
ARGS.crop = True
ARGS.stretch = True
ARGS.n_coords_sample = -1

shape = [24, 128, 128]
window = scroll_through_output(train_dl, shape=shape, transform=False)
# window = scroll_through_output(val_dl, shape=shape)
# window = scroll_through_output(test_dl, shape=shape)

In [ ]:
def gradient(y, x, grad_outputs=None):
    if grad_outputs is None:
        grad_outputs = torch.ones_like(y)
    grad = torch.autograd.grad(y, [x], grad_outputs=grad_outputs, create_graph=True)[0]
    return grad

In [ ]:
# criterion = nn.MSELoss()

# for idx, subj, proj, pcmra, coords, pcmra_array, mask_array in val_dl:
#     if idx == 1:
#         break

# print(pcmra.shape)
# out = get_complete_image(models, pcmra, coords, ARGS, output="pcmra")
# out = out.view(1, 1, 64, 64, 24).permute(0, 1, 4, 2, 3)

# plt.imshow(pcmra[0, 0, 8, :, :].cpu())
# plt.show()

# plt.imshow(out[0, 0, 8, :, :].cpu().detach())
# plt.show()

# print(criterion(pcmra, out).item())

In [ ]:
a = torch.randn(24, 1, 24, 128, 128).cuda()
print(a)


In [ ]:
t = time.time()

a = torch.floor(torch.round(a*10 ) / 10)

print(time.time() - t)


In [ ]:
print(a)